In [2]:
!pip install hyperopt

In [15]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboad_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train, 
            y_train,
            batch_size=params_fit.get('batch_size', 128),
            epochs=params_fit.get('epochs', 5),
            verbose=params_fit.get('verbose', 1),
            validation_data=params_fit.get('validation_data', (X_train, y_train)),
            callbacks=[tensorboad_callback]

            )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test,scoring = accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax( y_pred_prob, axis=1 )

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),               
  ])

In [11]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 2.7826 - accuracy: 0.2385 - val_loss: 1.4646 - val_accuracy: 0.5332
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 1.2125 - accuracy: 0.6086 - val_loss: 0.4583 - val_accuracy: 0.8570
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.5900 - accuracy: 0.8113 - val_loss: 0.1732 - val_accuracy: 0.9501
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.3656 - accuracy: 0.8867 - val_loss: 0.0918 - val_accuracy: 0.9705
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2567 - accuracy: 0.9206 - val_loss: 0.0612 - val_accuracy: 0.9811


0.9578231292517007

In [13]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.2024 - accuracy: 0.9578


0.9578231573104858

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),               
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(X_train, 
            y_train,
            batch_size=int(params.get('batch_size', 128)),
            epochs=5,
            verbose=0
            )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [29]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),

    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 140.0, 'dropout_cnn_block_one': 0.3726089356619093, 'dropout_cnn_block_three': 0.3887603115650262, 'dropout_cnn_block_two': 0.32587355456984474, 'dropout_dense_block_one': 0.46757350302541206, 'dropout_dense_block_two': 0.4856297086251842}
accuracy=0.94399094581604
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.3781987419896559, 'dropout_cnn_block_three': 0.42968866753548224, 'dropout_cnn_block_two': 0.4559387523725652, 'dropout_dense_block_one': 0.6699527109376988, 'dropout_dense_block_two': 0.6160245329588955}
accuracy=0.8501133918762207
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.40560503196842373, 'dropout_cnn_block_three': 0.43594831240777077, 'dropout_cnn_block_two': 0.34496085802166926, 'dropout_dense_block_one': 0.31976419201956535, 'dropout_dense_block_two': 0.5076389873564212}
accuracy=0.9106575846672058
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.3906053763545099, 'dropout_cnn_block_three': 0.3844112946663407, 'dropout_cnn_block_two': 0.34877355535